In [2]:
import pandas as pd
import numpy as np
import gzip
import xarray as xr
from datetime import datetime, timedelta
from calendar import monthrange
from netCDF4 import Dataset, date2num, num2date
import numpy.ma as ma

## Union GsMap

## Union to year .nc data

In [4]:
#longitude
lon = np.linspace(0.05,350.95,3600).tolist()
#latitude
lat = np.linspace(59.95,-59.95,1200).tolist()

st_year = 2023
ed_year = 2023

lists = ['0']
#year
for yr in range (st_year, ed_year+1) :
    lists = ['0']
    print ('Combine all files GSMAP........'+str(yr))
    path='D:/Rahma/GsMap/'+str(yr)+'/'
    #month
    for mt in range (1,6) :
        mti = format(mt,'02d')
        jday = monthrange(yr,mt)[1] 
        
        #day
        for dy in range (1,jday+1) :
            day = format(dy,'02d')
            
            #read file name
            filename = path+'gsmap_nrt.'+str(yr)+mti+day+'.0.1d.daily.00Z-23Z.dat.gz'
            gz = gzip.GzipFile(filename,'rb')
            
            #precipitation
            dd = np.frombuffer(gz.read(),dtype=np.float32)
            pre = dd.reshape([1200,3600])
            pre1 = pre.reshape([1,1200,3600])
            
            #time
            time = pd.date_range(str(yr)+"-"+mti+"-"+day, periods=1)
            
            #xarray file
            globals()[str(yr)+mti+day] = xr.Dataset(
                                            data_vars=dict(
                                                precipitation=(["time", "lat", "lon"], pre1),
                                            ),
                                            coords=dict(
                                                lon=(lon),
                                                lat=(lat),
                                                time=time,
       
                                            ),
                                            attrs=dict(description="GsMap 0.1 Degree"),
                                        )
            globals()['X'+str(yr)+mti+day]=globals()[str(yr)+mti+day].sel(lat=slice(6,-11), lon=slice(94,142))
            
            
            lists = lists + ['X'+str(yr)+mti+day]
            
            
            print ('read.....' + filename)
    

    print ('combine file GSMAP....'+str(yr))
    globals() ['GSMAP'+str(yr)] = globals()[lists[1]]
    for i in range (2, len(lists)) :
        globals() ['GSMAP'+str(yr)] = xr.combine_nested([globals() ['GSMAP'+str(yr)],globals()[lists[i]]], concat_dim=["time"]) 
        
    
    print ('save file GSMAP..........'+str(yr))
    globals() ['GSMAP'+str(yr)].to_netcdf('D:/Rahma/GsMap/combine/GSMAP'+str(yr)+'.nc')


Combine all files GSMAP........2023
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230101.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230102.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230103.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230104.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230105.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230106.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230107.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230108.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230109.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230110.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230111.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230112.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230113.0.1d.daily.00Z-23Z.dat.gz
re

read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230422.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230423.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230424.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230425.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230426.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230427.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230428.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230429.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230430.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230501.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230502.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230503.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt.20230504.0.1d.daily.00Z-23Z.dat.gz
read.....D:/Rahma/GsMap/2023/gsmap_nrt

## Get Longitude and Latitude for Thiessen

In [ ]:
data = Dataset('D:/Rahma/GsMap/combine/GSMAP2000.nc')
data

In [ ]:
long=data.variables['lon'][:]
lats=data.variables['lat'][:]
time=data.variables['time'][:]
lon = ma.getdata(long)
lat = ma.getdata(lats)
times = ma.getdata(time)
lon.shape, lat.shape, times.shape

In [ ]:
latmin = 0
latmax = lat.shape[0]
lonmin = 0
lonmax = lon.shape[0]

In [ ]:
lonarray= np.array([])
latarray = np.array ([])
lonindex = np.array([])
latindex = np.array([])

for i in range (latmin,latmax) :
    for j in range (lonmin,lonmax) :
        lons = lon[j]
        lats = lat[i]
        lon_i= j
        lat_i= i
        lonindex = np.append(lonindex,lon_i)
        latindex = np.append(latindex,lat_i)
        lonarray = np.append(lonarray,lons)
        latarray = np.append(latarray,lats)

In [ ]:
lonarray.shape, latarray.shape, latindex.shape, lonindex.shape

In [ ]:
lonlat = pd.DataFrame({'longitude':lonarray,'latitude':latarray,'lon_index':lonindex,'lat_index':latindex},columns=['longitude','latitude','lon_index','lat_index'])

In [ ]:
lonlat

In [ ]:
#Export dataframe to csv
lonlat.to_csv('D:/Rahma/SW Bilibili/RAAT/Hasil/grid_GsMap.csv',index=False)